In [28]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import ipaddress
from user_agents import parse
from sklearn.preprocessing import OneHotEncoder

In [29]:
dataset = pd.read_csv("../../dataset/Doceree-HCP_Train.csv", encoding="latin-1")

In [30]:
keyword_series = dataset["KEYWORDS"]

In [31]:
url_series = dataset["URL"]

In [32]:
url_series[0]

'https://www.cancertherapyadvisor.com/home/cancer-topics/prostate-cancer/'

In [33]:
from urllib.parse import urlparse

def extract_url_info(url):
    # Parse the URL
    parsed_url = urlparse(url)
    
    # Extract relevant information
    scheme = parsed_url.scheme
    domain = parsed_url.netloc
    path = parsed_url.path
    query = parsed_url.query
    fragment = parsed_url.fragment
    
    # Return the extracted information as a dictionary
    url_info = {
        "scheme": scheme,
        "domain": domain,
        "path": path,
        "query": query,
        "fragment": fragment
    }
    
    return url_info

# Example usage
url = url_series[47]
url_info = extract_url_info(url)

# Print the extracted information
for key, value in url_info.items():
    print(f"{key}: {value}")


scheme: https
domain: www.psychiatryadvisor.com
path: /home/topics/mood-disorders/depressive-disorder/medication-has-little-effect-emotional-blunting-major-depressive-disorder/
query: 
fragment: 


In [34]:
import tldextract

def extract_url_info(url):
    # Extract domain, subdomain, and suffix from the URL
    extracted = tldextract.extract(url)
    
    # Extract path, query, and fragment from the URL
    parsed_url = urlparse(url)
    path = parsed_url.path
    query = parsed_url.query
    fragment = parsed_url.fragment
    url_depth = len(path.split("/"))
    
    # Return the extracted information as a dictionary
    url_info = {
        "scheme": parsed_url.scheme,
        "subdomain": extracted.subdomain,
        "domain": extracted.domain,
        "path": path,
        "path-depth": url_depth,
        "length": len(path),
        "query": query,
        "fragment": fragment
    }
    
    return url_info

# # Example usage
# url = url_series[65132]
# url_info = extract_url_info(url)

# # Print the extracted information
# for key, value in url_info.items():
#     print(f"{key}: {value}")


In [35]:
url_infos = url_series.apply(extract_url_info)

In [36]:
df_url_info = pd.DataFrame(list(url_infos))

In [37]:
df_url_info.head()

,scheme,subdomain,domain,path,path-depth,length,query,fragment
0,https,www,cancertherapyadvisor,/home/cancer-topics/prostate-cancer/,5,36,,
1,https,www,cancertherapyadvisor,/home/decision-support-in-medicine/hematology/...,6,69,,
2,https,www,cancertherapyadvisor,/home/cancer-topics/prostate-cancer/,5,36,,
3,https,,globalrph,/medcalcs/warfarin-maintenance-dose-consult-tool/,4,49,,
4,https,www,cureus,/articles/58184-a-review-of-trikafta-triple-cy...,3,90,,


In [38]:
for col in df_url_info.columns:
    print(col, "====>", len(df_url_info[col].unique()))

scheme ====> 1
subdomain ====> 4
domain ====> 33
path ====> 2719
path-depth ====> 9
length ====> 197
query ====> 2569
fragment ====> 127


In [39]:
df_url_info["query"].isna()

0         False
1         False
2         False
3         False
4         False
          ...  
113932    False
113933    False
113934    False
113935    False
113936    False
Name: query, Length: 113937, dtype: bool

In [40]:
keyword_series

0         General|Endocrine|False|Medicine|Surgery|Urolo...
1         Bone Marrow|Radiography|Chronic|Oncology|Psych...
2         General|Endocrine|False|Medicine|Surgery|Urolo...
3         Dental|Total|Clinical|Pharmacology|Physicians|...
4         Critical Care|Emergency Medicine|General Pract...
                                ...                        
113932    Cardiovascular|Clinical|False|Male|Oncology|Ps...
113933    Asthma|Arthritis|General|Clinical|Diagnosis|Rh...
113934    Rehabilitation|Transplantation|Clinical|Preven...
113935    pain|disease|autoimmune|multiple sclerosis|gra...
113936    Genetic|Large|Autoimmune|Health|False|Abortion...
Name: KEYWORDS, Length: 113937, dtype: object

In [41]:
def split_keyword(value):
    value = value.split("|")
    return value

In [42]:
keyword_list = keyword_series.apply(split_keyword)

In [43]:
keyword_list

0         [General, Endocrine, False, Medicine, Surgery,...
1         [Bone Marrow, Radiography, Chronic, Oncology, ...
2         [General, Endocrine, False, Medicine, Surgery,...
3         [Dental, Total, Clinical, Pharmacology, Physic...
4         [Critical Care, Emergency Medicine, General Pr...
                                ...                        
113932    [Cardiovascular, Clinical, False, Male, Oncolo...
113933    [Asthma, Arthritis, General, Clinical, Diagnos...
113934    [Rehabilitation, Transplantation, Clinical, Pr...
113935    [pain, disease, autoimmune, multiple sclerosis...
113936    [Genetic, Large, Autoimmune, Health, False, Ab...
Name: KEYWORDS, Length: 113937, dtype: object

In [44]:
def convert_into_lower(value):
    value = [word.lower() for word in value]
    return value

In [45]:
keyword_list = keyword_list.apply(convert_into_lower)

In [55]:
keyword_list[0]

['general',
 'endocrine',
 'false',
 'medicine',
 'surgery',
 'urology',
 'chronic',
 'gastroenterology',
 'neurology',
 'physicians',
 'diagnosis',
 'psychiatry',
 'lung',
 'clinical',
 'dermatology',
 'ophthalmology',
 'cardiovascular',
 'cardiology',
 'oncology',
 'rheumatology']

In [57]:
# Flatten the list of lists into a single list
all_keywords = [keyword.lower() for sublist in keyword_list for keyword in sublist]

# Count the total unique keywords
unique_keywords = len(set(all_keywords))



# Print the count of unique keywords
print("Total unique keywords:", unique_keywords)


Total unique keywords: 19


In [62]:
all_keywords[1]

'e'

In [54]:
from gensim.models import Word2Vec

# Example list of words
word_list = ['apple', 'banana', 'orange']

# Train Word2Vec model on a corpus of text data
# You can replace this step with your own trained Word2Vec model
# or use pre-trained models available in gensim or other libraries
corpus = [['apple', 'fruit'], ['banana', 'fruit'], ['orange', 'fruit']]
model = Word2Vec(corpus, min_count=1, vector_size=100)

# Function to convert a list of words into a dense numerical vector
def get_vector(word_list):
    # Initialize an empty vector
    vector = [0.0] * model.vector_size

    # Iterate over each word in the list
    for word in word_list:
        # Check if the word is present in the Word2Vec model's vocabulary
        if word in model.wv:
            # Add the word's vector representation to the overall vector
            vector = [v + w for v, w in zip(vector, model.wv[word])]

    return vector

# Get the dense numerical vector for the word list
vector = get_vector(word_list)

# Print the vector
print(len(vector))


100


In [56]:
from gensim.models import Word2Vec

# Example list of keywords
keyword_list = ['general', 'endocrine', 'false', 'medicine', 'surgery', 'urology', 'chronic', 'gastroenterology',
                'neurology', 'physicians', 'diagnosis', 'psychiatry', 'lung', 'clinical', 'dermatology',
                'ophthalmology', 'cardiovascular', 'cardiology', 'oncology', 'rheumatology']

# Train Word2Vec model on a corpus of text data
# You can replace this step with your own trained Word2Vec model
# or use pre-trained models available in gensim or other libraries
corpus = [['general', 'endocrine', 'medicine', 'physicians'],
          ['surgery', 'urology', 'physicians', 'diagnosis'],
          ['chronic', 'gastroenterology', 'physicians'],
          ['neurology', 'physicians'],
          ['psychiatry', 'physicians', 'diagnosis'],
          ['lung', 'medicine', 'clinical'],
          ['dermatology', 'ophthalmology'],
          ['cardiovascular', 'cardiology'],
          ['oncology'],
          ['rheumatology', 'diagnosis']]
model = Word2Vec(corpus, min_count=1, vector_size=100)

# Function to convert a list of keywords into a dense numerical vector
def get_vector(keyword_list):
    # Initialize an empty vector
    vector = [0.0] * model.vector_size

    # Iterate over each keyword in the list
    for keyword in keyword_list:
        # Check if the keyword is present in the Word2Vec model's vocabulary
        if keyword in model.wv:
            # Add the keyword's vector representation to the overall vector
            vector = [v + w for v, w in zip(vector, model.wv[keyword])]

    return vector

# Get the dense numerical vector for the keyword list
vector = get_vector(keyword_list)

# Print the vector
print(vector)


[-0.012847653037169948, 0.024732093239435926, 0.014773901624721475, 0.004695273062679917, 0.021751812571892515, -0.038580811640713364, 0.021702515776269138, 0.06737657165831479, -0.046380454674363136, -0.04900260943395551, 0.0040979764016810805, -0.05156094909762032, -0.01938246248755604, 0.0019869895186275244, 0.01534041432023514, -0.004738153045764193, 0.030903884558938444, 0.015400988249893999, -0.003778961719945073, -0.0814630888053216, 0.020154652554992936, 0.0012496845447458327, 0.055699559050140124, -0.017330223752651364, -0.012212980262120254, 0.02085068030282855, -0.034109779342543334, 0.01740062842145562, -0.005788701178971678, 0.009412195824552327, 0.024990404024720192, -0.022140714863780886, 0.010542724368860945, -0.02519330427821842, -0.007802132113283733, 0.023078149184584618, 0.017191444436321035, 0.017090452420234215, -0.018475067867257167, -0.012905889190733433, 0.03364587691612542, -0.03417778690345585, -0.019791923434240744, 0.04401062341639772, 0.0170227242524561, -